In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
from pathlib import Path
import os
import rasterio
from shapely.geometry import Point, Polygon
from rasterstats import zonal_stats
from rasterio.plot import show_hist
# import rioxarray as rxr
from rasterio.mask import mask
from rasterio import plot as rioplot
import matplotlib.pyplot as plt
import contextily as ctx
import ee
# import geemap
import fiona
# import geemap.colormaps as cm
import rasterio
from rasterio.plot import show
from shapely.geometry import Point, Polygon

In [ ]:
data_folder = Path('../Data/raw')
databases = Path('/Users/david/Dropbox/PhD/Data/')

In [ ]:
cantons_ch = gpd.read_file('/Users/david/Dropbox/PhD/Data/Databases/Shapefiles/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
cantons_ch = cantons_ch.to_crs(21781)

In [ ]:
statpop = pd.read_csv('/Users/david/Dropbox/PhD/Data/Databases/OFS/ag-b-00.03-vz2020statpop/STATPOP2020.csv',sep = ';')
statpop_ha = statpop.copy()
geometry = [Point(xy) for xy in zip(statpop['E_KOORD'], statpop['N_KOORD'])]
statpop_point = gpd.GeoDataFrame(statpop, crs=2056, geometry=geometry)   

In [ ]:
geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(statpop_ha.E_KOORD, statpop_ha.N_KOORD)]
statpop_ha = gpd.GeoDataFrame(statpop_ha, crs=2056, geometry=geometry)      

In [ ]:
df_swiss_sep1 = gpd.read_file(data_folder/'Swiss-SEP'/'SNC_Swiss-SEP1'/'SHP'/'ssep_user_geo.shp', driver = 'Shapefile')

df_swiss_sep2 = gpd.read_file(data_folder/'Swiss-SEP'/'SNC_Swiss-SEP2'/'SHP'/'ssep2_user_geo.shp', driver = 'Shapefile')

In [ ]:
country_geo = gpd.read_file("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2l15.shp")
with fiona.open("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2l15.shp", "r") as shapefile:
    country_geo_fiona = [feature["geometry"] for feature in shapefile]

In [ ]:
lakes = gpd.read_file("/Users/david/Dropbox/PhD/GitHub/COVID19/input/g2s15.shp")

In [ ]:
df_swiss_sep2

In [ ]:
lakes[lakes.GMDNAME.isin(['Lac Léman'])].to_file('/Users/david/Dropbox/PhD/GitHub/GEOCOVID-phase2/data/lake_vd.shp', driver = "Shapefile")

In [ ]:
# Open the file:
relief_raster = rasterio.open('/Users/david/Dropbox/PhD/GitHub/COVID19/input/02-relief-ascii.asc')

# Check type of the variable 'raster'
type(relief_raster)

In [ ]:
out_image, out_transform = rasterio.mask.mask(relief_raster, country_geo_fiona, crop=True, filled=False)
out_meta = relief_raster.meta

In [ ]:
array = relief_raster.read(1)

In [ ]:
show(array, cmap='Greys_r')

In [ ]:
relief_raster.crs

In [ ]:
show(out_image, cmap='Greys_r')

In [ ]:
ch_extent = np.asarray(country_geo.bounds)[0][[0,2,1,3]]

In [ ]:
ax = df_swiss_sep2.plot('ssep2_d', markersize = 0.5, cmap = 'RdYlGn', legend = True,figsize = (15, 15), legend_kwds = {'shrink':0.5})
lakes.plot(color = 'lightblue', ax=ax)
cantons_ch.geometry.boundary.plot(ax=ax,edgecolor='k', color=None, linewidth=0.1)
plt.imshow(out_image.squeeze(),extent=ch_extent, cmap='Greys_r', alpha=0.4)
ax.set_axis_off()

In [ ]:
ax = df_swiss_sep2.plot('ssep2_t', markersize = 0.5, cmap = 'RdYlGn', legend = True,figsize = (15, 15), legend_kwds = {'shrink':0.5})
lakes.plot(color = 'lightblue', ax=ax)
cantons_ch.geometry.boundary.plot(ax=ax,edgecolor='k', color=None, linewidth=0.1)
plt.imshow(out_image.squeeze(),extent=ch_extent, cmap='Greys_r', alpha=0.4)
ax.set_axis_off()